In [ ]:
import pickle
import pandas as pd
import os
import numpy as np

In [2]:
dataN=pd.read_csv('../data/nyc_filter_data.csv')
dataT=pd.read_csv('../data/tky_filter_data.csv')
print(dataT.columns)

Index(['userid', 'venueid', 'catid', 'catname', 'latitute', 'longitude',
       'timezone', 'time', 'categid', 'timestamp', 'hour', 'day', 'week',
       'hour_48'],
      dtype='object')


In [4]:
#產生序列資料   長短不限需要padding  至少需要3
def pad_data(data,n):
    seqlen=n
    dataList=[]
    group=data.groupby('userid')
    for cid,i in group:
        i=i.sort_values(by='timestamp')
        seq=[]
        nowseq=[]
        nowtime=0
        pasttime=0
        for z,j in i.iterrows():
            nowtime=j['timestamp']
            #序列中每個簽到只取以下幾個特徵
            nowseq=[j['userid'],j['venueid'],j['categid'],j['latitute'],j['longitude'],j['hour'],j['hour_48'],j['week'],j['timestamp']]
            tint=nowtime-pasttime
            if len(seq)>=15:                               #設定序列最長為15
                dataList.append(seq)
                seq=[]
                seq.append(nowseq)
                pasttime=nowtime
            elif pasttime==0 or tint<60*60*24:             #超過1天則斷開
                seq.append(nowseq)
                pasttime=nowtime
            else:
                if len(seq)>=seqlen:
                    dataList.append(seq)
                seq=[]
                seq.append(nowseq)
                pasttime=nowtime
        if len(seq)>=seqlen:
            dataList.append(seq)
    print("序列數:",len(dataList))
    return dataList


pad_N=pad_data(dataN,3)
with open('../data/nyc_seq_data_pad', 'wb') as fp:
    pickle.dump(pad_N, fp)
pad_T=pad_data(dataT,3)
with open('../data/tky_seq_data_pad', 'wb') as fp:
    pickle.dump(pad_T, fp)

序列數: 15992
序列數: 38360


In [10]:
#平均序列長度
sumlen=0
c=0
for i in pad_N:
    #print(i)
    #print(len(i))
    sumlen+=len(i)
    c+=1
print(sumlen/c)

6.5580915457728866
